In [ ]:
import pandas as pd
import numpy as np
import itertools
import requests as rq
from bs4 import BeautifulSoup as BS

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'

In [ ]:
## we choose the data columns we want from the dataset, in order to save storage
words_count = ['summary','space','description','neighborhood_overview','notes',\
               'transit','access','interaction','house_rules']

house_attributes = ['property_type', 'room_type',\
                    'accommodates', 'bathrooms', 'bedrooms', 'beds','amenities']

host_attributes_1 = ['host_id','host_is_superhost']

host_attributes_2 = ['host_id','host_response_time', 'host_response_rate', 'host_acceptance_rate',\
                    'host_identity_verified']

review_1 = ['number_of_reviews','review_scores_rating']

review_2 = ['review_scores_accuracy', 'review_scores_checkin','review_scores_cleanliness',\
            'review_scores_communication','review_scores_location', 'review_scores_rating',\
            'review_scores_value']

policies = ['instant_bookable','cancellation_policy','minimum_nights']

location_ = ['latitude','longitude']

df_y = ['price']

columns = [['id'],words_count, host_attributes_1, host_attributes_2,\
           review_1,review_2,house_attributes,policies,location_,df_y]

columns_m =  list(itertools.chain.from_iterable(columns)) 
columns_ = list(set(columns_m))

## store the variables
%store words_count 
%store host_attributes_1
%store host_attributes_2
%store review_1
%store review_2
%store house_attributes
%store policies
%store location_

In [ ]:
insideairbnb = 'http://insideairbnb.com/get-the-data.html'
r = rq.get(insideairbnb)

def _getUrls_(res):  ## getUrls from the html page
    hrefs = []
    soup = BS(res.text, 'html.parser')
    main_content = soup.find(class_='table table-hover table-striped beijing')
    table = main_content.findAll('a',href=True)
    listingnum = [i for i in range(len(table)) if i % 7 == 0]
    for i in listingnum:
        url = str(table[i]).split('" onclick="var')[0][9:]
        hrefs.append(url)
    return(hrefs)

### Method1 

In [ ]:
## read and store 17 files, but took too many storage, therefore we used method 2
for url in _getUrls_(r):
    r = rq.get(url)
    url_parts = url.split('/')
    file_name = url_parts[-1].split('.')[0]+'-'+url_parts[-3]
    if r.status_code == 200:
        with open('{}.zip.gz'.format(file_name),'wb') as f:
            f.write(r.content)

### Method2

In [ ]:
#read url directly from the website
for url in _getUrls_(r):
    try:
        if url == 'http://data.insideairbnb.com/china/beijing/beijing/2019-09-23/data/listings.csv.gz': 
            df_listings = pd.read_csv(url,error_bad_lines=False, index_col=False, dtype='unicode',usecols =columns_)
            df_listings['year'] = url[-31:-21]
        else:
            df = pd.read_csv(url,error_bad_lines=False, index_col=False, dtype='unicode',usecols =columns_)
            df['year'] = url[-31:-21]
            df_listings = df_listings.append(df)
    except Exception as e:
        print(e)


In [ ]:
## explore the files as a csv file
df_listings.to_csv('listing_1_yr_beijing.csv')